In [1]:
!pip install networkx

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import sys
import os
import networkx as nx
from collections import Counter
project_path = os.getcwd().split("Code")[0]
sys.path.append(project_path)

# Importa la classe GraphConstructor
from Code.notebook.graph.GraphConstructor import GraphConstructor

# Funzione per eseguire il clustering con Label Propagation
def label_propagation_clustering(graph):
    """
    Esegue il clustering sul grafo utilizzando l'algoritmo di Label Propagation.
    
    Args:
        graph (networkx.Graph): Il grafo su cui eseguire il clustering.
    
    Returns:
        dict: Un dizionario con l'etichetta del cluster per ciascun nodo.
    """
    # Converti il grafo in un grafo non diretto
    undirected_graph = graph.to_undirected()
    
    # Applica l'algoritmo di Label Propagation
    communities = nx.algorithms.community.label_propagation_communities(undirected_graph)
    
    # Crea un dizionario dei cluster associando i nodi alle comunità
    cluster_labels = {}
    for cluster_id, community in enumerate(communities):
        for node in community:
            cluster_labels[node] = cluster_id
            
    return cluster_labels

# Funzione per sanitizzare gli attributi del grafo (nodi e archi)
def sanitize_graph_attributes(graph):
    """
    Sanitizza le chiavi e i valori degli attributi dei nodi e degli archi
    per renderli compatibili con il formato GML.
    """
    for node in graph.nodes:
        attrs = graph.nodes[node]
        sanitized_attrs = {}
        for key, value in attrs.items():
            # Rimpiazza spazi e trattini nei nomi delle chiavi
            new_key = str(key).replace(" ", "_").replace("-", "_")
            # Converte tutti i valori in stringhe
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.nodes[node].clear()
        graph.nodes[node].update(sanitized_attrs)

    for u, v, attrs in graph.edges(data=True):
        sanitized_attrs = {}
        for key, value in attrs.items():
            # Rimpiazza spazi e trattini nei nomi delle chiavi
            new_key = str(key).replace(" ", "_").replace("-", "_")
            # Converte tutti i valori in stringhe
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.edges[u, v].clear()
        graph.edges[u, v].update(sanitized_attrs)

# Crea il grafo utilizzando GraphConstructor
graph_builder = GraphConstructor()
graph_builder.build_graph()
graph = graph_builder.graph

# Esegui il clustering con Label Propagation
lp_labels = label_propagation_clustering(graph)

# Copia il grafo originale per assegnare i cluster
gr = graph.copy()

# Assegna il cluster a ciascun nodo
for node, label in lp_labels.items():
    gr.nodes[node]['cluster'] = label  # Assegniamo l'etichetta del cluster al nodo

# Sanitizza gli attributi del grafo prima di esportarlo
sanitize_graph_attributes(gr)

# Esporta il grafo in formato GML
output_path = "graph_with_label_propagation_clusters.gml"
nx.write_gml(gr, output_path)
